# Delta Lake - Cidades Brasileiras

Este notebook demonstra a criação, leitura e manipulação de uma tabela Delta usando o dataset `cidades_brasileiras.csv`.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType

from delta import *

import logging

logging.getLogger("py4j").setLevel(logging.DEBUG)

In [2]:
spark = (
    SparkSession
    .builder
    .master("local[*]")
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.2.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .getOrCreate()
)

25/04/24 11:37:12 WARN Utils: Your hostname, edsatc resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/04/24 11:37:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ed/.cache/pypoetry/virtualenvs/eng-dados-spark-EC3tKTXJ-py3.11/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ed/.ivy2/cache
The jars for the packages stored in: /home/ed/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-89a93d06-a9e0-4e0c-bd50-0f4780420911;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 298ms :: artifacts dl 16ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.0 from central in [default]
	io.delta#delta-storage;3.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0

In [3]:
schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("cidade", StringType(), True),
    StructField("estado", StringType(), True),
    StructField("sigla", StringType(), True),
    StructField("ibge", IntegerType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True)
])

In [4]:
df = spark.read.csv("../data/cidades_brasileiras.csv", header=True, schema=schema)
df.show(5)

+---+-------------------+------------+-----+-------+----------+----------+
| id|             cidade|      estado|sigla|   ibge|  latitude| longitude|
+---+-------------------+------------+-----+-------+----------+----------+
|  1|    ABADIA DE GOIÁS|       GOIÁS|   GO|5200050|-16.757264| -49.44122|
|  2|ABADIA DOS DOURADOS|MINAS GERAIS|   MG|3100104|-18.491063|-47.406365|
|  3|          ABADIÂNIA|       GOIÁS|   GO|5200100|-16.194723|-48.706812|
|  4|             ABAETÉ|MINAS GERAIS|   MG|3100203|-19.156683|-45.448121|
|  5|         ABAETETUBA|        PARÁ|   PA|1500107| -1.721828|-48.878843|
+---+-------------------+------------+-----+-------+----------+----------+
only showing top 5 rows



In [5]:
df.write.format("delta").mode("overwrite").saveAsTable("cidades_delta")
spark.sql("DROP TABLE IF EXISTS cidades_delta")
spark.sql("CREATE TABLE cidades_delta USING DELTA LOCATION '../output/delta/cidades'")

DataFrame[]

In [6]:
spark.sql("SHOW TABLES").show()



+---------+-------------+-----------+
|namespace|    tableName|isTemporary|
+---------+-------------+-----------+
|  default|cidades_delta|      false|
+---------+-------------+-----------+



In [7]:
spark.sql("SELECT * FROM cidades_delta").show(5)


25/04/24 11:37:57 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---+-------------------+------------+-----+-------+----------+----------+
| id|             cidade|      estado|sigla|   ibge|  latitude| longitude|
+---+-------------------+------------+-----+-------+----------+----------+
|  1|    ABADIA DE GOIÁS|       GOIÁS|   GO|5200050|-16.757264| -49.44122|
|  2|ABADIA DOS DOURADOS|MINAS GERAIS|   MG|3100104|-18.491063|-47.406365|
|  3|          ABADIÂNIA|       GOIÁS|   GO|5200100|-16.194723|-48.706812|
|  4|             ABAETÉ|MINAS GERAIS|   MG|3100203|-19.156683|-45.448121|
|  5|         ABAETETUBA|        PARÁ|   PA|1500107| -1.721828|-48.878843|
+---+-------------------+------------+-----+-------+----------+----------+
only showing top 5 rows



In [8]:
spark.sql("""
    INSERT INTO cidades_delta VALUES
    (9999, 'Cidade Exemplo', 'Estado Exemplo', 'EX', 9999999, -10.1234, -50.5678)
""")

spark.sql("SELECT * FROM cidades_delta WHERE id = 9999").show()


+----+--------------+--------------+-----+-------+--------+---------+
|  id|        cidade|        estado|sigla|   ibge|latitude|longitude|
+----+--------------+--------------+-----+-------+--------+---------+
|9999|Cidade Exemplo|Estado Exemplo|   EX|9999999|-10.1234| -50.5678|
+----+--------------+--------------+-----+-------+--------+---------+



In [9]:
spark.sql("""
    UPDATE cidades_delta
    SET latitude = -11.0000, longitude = -51.0000
    WHERE id = 9999
""")

spark.sql("SELECT * FROM cidades_delta WHERE id = 9999").show()

+----+--------------+--------------+-----+-------+--------+---------+
|  id|        cidade|        estado|sigla|   ibge|latitude|longitude|
+----+--------------+--------------+-----+-------+--------+---------+
|9999|Cidade Exemplo|Estado Exemplo|   EX|9999999|   -11.0|    -51.0|
+----+--------------+--------------+-----+-------+--------+---------+



In [10]:
spark.sql("""
    DELETE FROM cidades_delta
    WHERE id = 9999
""")

spark.sql("SELECT * FROM cidades_delta WHERE id = 9999").show()


+---+------+------+-----+----+--------+---------+
| id|cidade|estado|sigla|ibge|latitude|longitude|
+---+------+------+-----+----+--------+---------+
+---+------+------+-----+----+--------+---------+



In [11]:
df_final = spark.read.format("delta").load("../output/delta/cidades")
df_final.show()

+---+-------------------+--------------+-----+-------+----------+----------+
| id|             cidade|        estado|sigla|   ibge|  latitude| longitude|
+---+-------------------+--------------+-----+-------+----------+----------+
|  1|    ABADIA DE GOIÁS|         GOIÁS|   GO|5200050|-16.757264| -49.44122|
|  2|ABADIA DOS DOURADOS|  MINAS GERAIS|   MG|3100104|-18.491063|-47.406365|
|  3|          ABADIÂNIA|         GOIÁS|   GO|5200100|-16.194723|-48.706812|
|  4|             ABAETÉ|  MINAS GERAIS|   MG|3100203|-19.156683|-45.448121|
|  5|         ABAETETUBA|          PARÁ|   PA|1500107| -1.721828|-48.878843|
|  6|            ABAIARA|         CEARÁ|   CE|2300101| -7.360781|-39.048788|
|  7|             ABAÍRA|         BAHIA|   BA|2900108| -13.25033|-41.664034|
|  8|              ABARÉ|         BAHIA|   BA|2900207| -8.723991|-39.113969|
|  9|             ABATIÁ|        PARANÁ|   PR|4100103|-23.305297|-50.310253|
| 10|      ABDON BATISTA|SANTA CATARINA|   SC|4200051|-27.612419|-51.022992|